In [383]:
import pandas as pd
import numpy as np
import requests
import time
from io import StringIO
from bs4 import BeautifulSoup
import re

Choose starting point url

In [440]:
url = "https://fbref.com/en/comps/"
data = requests.get(url)

In [479]:
desired_competition_ids = ['37','24','9','12','13','20','11','32','10']

Create a list of all the male competitions on FBREF

In [480]:
soup = BeautifulSoup(data.text)
table1 = soup.find("table", {"id": "comps_1_fa_club_league_senior"})
male_comps_table1 = table1.find_all("tr", {"class": "gender-m"})

table2 = soup.find("table", {"id": "comps_2_fa_club_league_senior"})
male_comps_table2 = table2.find_all("tr", {"class": "gender-m"})

male_comps = male_comps_table1 + male_comps_table2

AttributeError: 'NoneType' object has no attribute 'find_all'

In [481]:
# Debug: Print number of rows found
print(f"Total male_comps rows found: {len(male_comps)}")

# Filter rows based on desired competition IDs
filtered_male_comps = []
for row in male_comps:
    # Find the href attribute of the first <a> tag in the row
    a_tag = row.find("a", href=True)
    if a_tag:
        row_href = a_tag["href"]
        print(f"Found href: {row_href}")  # Debug: Print each href found
        # Extract competition ID using regex
        match = re.search(r'/en/comps/(\d+)/', row_href)
        if match:
            comp_id = match.group(1)
            print(f"Competition ID extracted: {comp_id}")  # Debug: Print competition ID
            if comp_id in desired_competition_ids:
                print(f"Match found for competition ID: {comp_id}")  # Debug: Print if match is found
                filtered_male_comps.append(row)

# Debug: Print number of rows after filtering
print(f"Filtered male_comps rows: {len(filtered_male_comps)}")

Total male_comps rows found: 63
Found href: /en/comps/21/history/Liga-Profesional-Argentina-Seasons
Competition ID extracted: 21
Found href: /en/comps/65/history/A-League-Men-Seasons
Competition ID extracted: 65
Found href: /en/comps/56/history/Austrian-Bundesliga-Seasons
Competition ID extracted: 56
Found href: /en/comps/37/history/Belgian-Pro-League-Seasons
Competition ID extracted: 37
Match found for competition ID: 37
Found href: /en/comps/74/history/Bolivian-Primera-Division-Seasons
Competition ID extracted: 74
Found href: /en/comps/24/history/Serie-A-Seasons
Competition ID extracted: 24
Match found for competition ID: 24
Found href: /en/comps/67/history/Bulgarian-First-League-Seasons
Competition ID extracted: 67
Found href: /en/comps/211/history/Canadian-Premier-League-Seasons
Competition ID extracted: 211
Found href: /en/comps/35/history/Primera-Division-Seasons
Competition ID extracted: 35
Found href: /en/comps/62/history/Chinese-Super-League-Seasons
Competition ID extracted: 6

In [496]:
filtered_male_comps

[<tr class="gender-m"><th class="left" data-stat="league_name" scope="row"><a href="/en/comps/37/history/Belgian-Pro-League-Seasons">Belgian Pro League</a></th><td class="center" data-stat="gender">M</td><td class="left" data-stat="country"><a href="/en/country/BEL/Belgium-Football"><span class="f-i f-be" style="">be</span></a> <a href="/en/country/BEL/Belgium-Football">BEL</a></td><td class="left" csk="2003-2004" data-stat="minseason"><a href="/en/comps/37/2003-2004/2003-2004-Belgian-Pro-League-Stats">2003-2004</a></td><td class="left" csk="2024-2025" data-stat="maxseason"><a href="/en/comps/37/Belgian-Pro-League-Stats">2024-2025</a></td></tr>,
 <tr class="gender-m"><th class="left" data-stat="league_name" scope="row"><a href="/en/comps/24/history/Serie-A-Seasons">Campeonato Brasileiro Série A</a></th><td class="center" data-stat="gender">M</td><td class="left" data-stat="country"><a href="/en/country/BRA/Brazil-Football"><span class="f-i f-br" style="">br</span></a> <a href="/en/coun

Loop through the rows and create a link for each competition

In [482]:
links = []
for row in filtered_male_comps:
    row_links = [l.get("href") for l in row.find_all("a") if '/comps/' in l.get("href")]
    if len(row_links) >= 2:
        links.append(row_links[0])
        
    comps_urls = [f"https://fbref.com{l}" for l in links]
    
    time.sleep(3)

In [483]:
comps_urls

['https://fbref.com/en/comps/37/history/Belgian-Pro-League-Seasons',
 'https://fbref.com/en/comps/24/history/Serie-A-Seasons',
 'https://fbref.com/en/comps/9/history/Premier-League-Seasons',
 'https://fbref.com/en/comps/12/history/La-Liga-Seasons',
 'https://fbref.com/en/comps/13/history/Ligue-1-Seasons',
 'https://fbref.com/en/comps/20/history/Bundesliga-Seasons',
 'https://fbref.com/en/comps/11/history/Serie-A-Seasons',
 'https://fbref.com/en/comps/32/history/Primeira-Liga-Seasons',
 'https://fbref.com/en/comps/10/history/Championship-Seasons']

Loop through all competition links to find each season you want. Then loop through every item in that season to create the urls for each season. Lastly loop through each url and create a stats version of that url which has the player stats

In [484]:
# Initialize the list to store the final stats URLs
season_stats_urls = []

for url in comps_urls:
    try:
        # Fetch the page content
        data = requests.get(url)
        soup = BeautifulSoup(data.text, 'html.parser')

        # Find the table with seasons
        seasons_table = soup.find("table", {"id": "seasons"})
        if not seasons_table:
            raise ValueError("Table with ID 'seasons' not found")
        
        # Find the first two relevant season rows (skipping the header)
        season_rows = seasons_table.find_all("tr")[1:3]

        for row in season_rows:
            # Extract links that contain '/comps/' in the href attribute
            row_links = [l.get("href") for l in row.find_all("a") if '/comps/' in l.get("href")]
            
            # If there are at least 2 such links, use the first one to construct the season URL
            if len(row_links) >= 2:
                season_url = f"https://fbref.com{row_links[0]}"
                
                # Modify the season URL to get the stats version
                stats_url = re.sub(r'(/[^/]+)$', r'/stats\1', season_url)
                
                # Add the stats URL to the final list
                season_stats_urls.append(stats_url)
        
        print(f"{url}: Done")
        
        # Optional delay between requests
        time.sleep(3)
    
    except Exception as e:
        print(f"{url}: {type(e).__name__} - {str(e)}")
        print(f"HTML content: {soup.prettify()}")

# Now you have all the season_stats_urls ready in one loop


https://fbref.com/en/comps/37/history/Belgian-Pro-League-Seasons: Done
https://fbref.com/en/comps/24/history/Serie-A-Seasons: Done
https://fbref.com/en/comps/9/history/Premier-League-Seasons: Done
https://fbref.com/en/comps/12/history/La-Liga-Seasons: Done
https://fbref.com/en/comps/13/history/Ligue-1-Seasons: Done
https://fbref.com/en/comps/20/history/Bundesliga-Seasons: Done
https://fbref.com/en/comps/11/history/Serie-A-Seasons: Done
https://fbref.com/en/comps/32/history/Primeira-Liga-Seasons: Done
https://fbref.com/en/comps/10/history/Championship-Seasons: Done


In [485]:
season_stats_urls

['https://fbref.com/en/comps/37/stats/Belgian-Pro-League-Stats',
 'https://fbref.com/en/comps/37/2023-2024/stats/2023-2024-Belgian-Pro-League-Stats',
 'https://fbref.com/en/comps/24/stats/Serie-A-Stats',
 'https://fbref.com/en/comps/24/2023/stats/2023-Serie-A-Stats',
 'https://fbref.com/en/comps/9/stats/Premier-League-Stats',
 'https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats',
 'https://fbref.com/en/comps/12/stats/La-Liga-Stats',
 'https://fbref.com/en/comps/12/2023-2024/stats/2023-2024-La-Liga-Stats',
 'https://fbref.com/en/comps/13/stats/Ligue-1-Stats',
 'https://fbref.com/en/comps/13/2023-2024/stats/2023-2024-Ligue-1-Stats',
 'https://fbref.com/en/comps/20/stats/Bundesliga-Stats',
 'https://fbref.com/en/comps/20/2023-2024/stats/2023-2024-Bundesliga-Stats',
 'https://fbref.com/en/comps/11/stats/Serie-A-Stats',
 'https://fbref.com/en/comps/11/2023-2024/stats/2023-2024-Serie-A-Stats',
 'https://fbref.com/en/comps/32/stats/Primeira-Liga-Stats',
 'https://fbre

In [486]:
all_dfs = []
for url in season_stats_urls:
    data = requests.get(url).text.replace("<!--", "").replace("-->", "")
    soup = BeautifulSoup(data)
    header = soup.find("h2").find("span").text
    season, comp_name = header.split(" ", 1) 
    country = soup.find("a", href=lambda href: href and '/country/' in href).text
    
    table = soup.find("table", {"id": "stats_standard"})
    ids = [x["data-append-csv"] for x in table.find_all("td", {"data-stat": "player"})]

    df = pd.read_html(StringIO(str(table)))[0]
    df.columns = [f"{i} {j}" if "Unnamed" not in i else j for i, j in df.columns]
    df = df.loc[df["Rk"] != "Rk"]
    df["Fbref"] = ids
    df['Season'] = season
    df['Competition'] = comp_name
    df['Country'] = country
    
    cols_to_move = ['Fbref', 'Season', 'Competition', 'Country']
    remaining_cols = [col for col in df.columns if col not in cols_to_move]
    new_order = remaining_cols[:1] + cols_to_move + remaining_cols[1:]
        
    df = df[new_order]
    df = df.rename({'Playing Time MP': 'MP','Playing Time Starts': 'Starts', 'Playing Time Min': 'Min', 'Playing Time 90s': '90s', 'Per 90 Minutes npxG': 'npxG', 'Per 90 Minutes xAG': 'xAG'}, axis=1)
    df = df.groupby(['Fbref', 'Season', 'Competition', 'Country', 'Player', 'Squad', 'Age']).agg(lambda x: pd.to_numeric(x, errors='coerce').mean())[['MP', 'Starts', 'Min', '90s', 'npxG', 'xAG']].reset_index()
    
    all_dfs.append(df)
    time.sleep(3)

AttributeError: 'NoneType' object has no attribute 'text'

In [487]:
all_players = pd.concat(all_dfs)

In [488]:
all_players.head(20)

,Fbref,Season,Competition,Country,Player,Squad,Age,MP,Starts,Min,90s,npxG,xAG
0,003d8899,2024-2025,Belgian Pro League,Belgium,Takahiro Akimoto,OH Leuven,26-205,4.0,4.0,360.0,4.0,0.02,0.02
1,028b8d59,2024-2025,Belgian Pro League,Belgium,Archie Brown,Gent,22-087,4.0,4.0,357.0,4.0,0.03,0.04
2,051e11ca,2024-2025,Belgian Pro League,Belgium,Alouis Diriken,Sint-Truiden,20-232,2.0,2.0,147.0,1.6,0.00,0.00
3,074e1710,2024-2025,Belgian Pro League,Belgium,Mike Penders,Genk,19-023,2.0,2.0,180.0,2.0,0.00,0.00
4,07caccd9,2024-2025,Belgian Pro League,Belgium,Amine Boukamir,Charleroi,17-305,1.0,0.0,11.0,0.1,0.00,0.00
5,098525c6,2024-2025,Belgian Pro League,Belgium,Tobe Leysen,OH Leuven,22-167,4.0,4.0,360.0,4.0,0.00,0.00
6,0a6dbbf7,2024-2025,Belgian Pro League,Belgium,Brandon Mechele,Club Brugge,31-208,4.0,4.0,360.0,4.0,0.03,0.02
7,0abcb968,2024-2025,Belgian Pro League,Belgium,Rafik Belghali,Mechelen,22-077,2.0,0.0,50.0,0.6,0.00,0.00
8,0beabf0a,2024-2025,Belgian Pro League,Belgium,Joaquin Seys,Club Brugge,19-148,4.0,3.0,235.0,2.6,0.08,0.12
9,0c317bc1,2024-2025,Belgian Pro League,Belgium,Malcolm Viltard,Dender,21-304,4.0,3.0,266.0,3.0,0.05,0.01


In [490]:
all_players['Season'].unique()

array(['2024-2025', '2023-2024', '2024', '2023'], dtype=object)

In [495]:
all_players.query('Fbref == "220c11f1"').sort_values(by= 'npxG' , ascending=False).head(20)

,Fbref,Season,Competition,Country,Player,Squad,Age,MP,Starts,Min,90s,npxG,xAG
50,220c11f1,2024-2025,La Liga,Spain,Cristhian Stuani,Girona,37-316,1.0,0.0,10.0,0.1,1.68,0.00
92,220c11f1,2023-2024,La Liga,Spain,Cristhian Stuani,Girona,36,31.0,5.0,804.0,8.9,1.08,0.21


In [210]:

def fetch_players(url):
    with requests.Session() as session:
        html = session.get(url).text.replace("<!--", "").replace("-->", "")

    soup = BeautifulSoup(html, "html.parser")
    table = soup.find("table", {"id": "stats_standard"})
    ids = [x["data-append-csv"] for x in table.find_all("td", {"data-stat": "player"})]

    df = pd.read_html(StringIO(str(table)))[0]
    df.columns = [f"{i} {j}" if "Unnamed" not in i else j for i, j in df.columns]
    df = df.loc[df["Rk"] != "Rk"]
    df["fbref"] = ids
    
    df = df['fbref','Player', 'Squad', 'Age', 'Playing Time MP', 'Playing Time Starts', 'Playing Time Min', 'Playing Time 90s', 'Per 90 Minutes npxG', 'Per 90 Minutes xAG']
    df = df.rename({'Playing Time MP': 'MP','Playing Time Starts': 'Starts', 'Playing Time Min': 'Min', 'Playing Time 90s': '90s', 'Per 90 Minutes npxG': 'npxG', 'Per 90 Minutes xAG': 'xAG'}, axis=1)
    df = df.groupby(['fbref','Player', 'Squad', 'Age']).agg(lambda x: pd.to_numeric(x, errors='coerce').mean())[['MP', 'Starts', 'Min', '90s', 'npxG', 'xAG']].reset_index()
    
    return df

In [211]:
prem_url = "https://fbref.com/en/comps/9/2023-2024/stats/2023-2024-Premier-League-Stats"
champ_url = "https://fbref.com/en/comps/10/2023-2024/stats/2023-2024-Championship-Stats"

In [212]:
prem_df = fetch_players(prem_url)
champ_df = fetch_players(champ_url)

In [215]:
champ_df['npxG'] = np.round(champ_df['npxG']*0.71, 2)
champ_df['xAG'] = np.round(champ_df['xAG']*0.71, 2)

In [216]:
players_df = pd.concat([prem_df, champ_df], ignore_index=True)

In [217]:
players_df

,fbref,Player,Squad,Age,MP,Starts,Min,90s,npxG,xAG
0,00242715,Moussa Niakhate,Nott'ham Forest,27,21.0,15.0,1460.0,16.2,0.07,0.01
1,003cf4d1,Jayden Danns,Liverpool,17,2.0,0.0,9.0,0.1,0.00,0.00
2,00953a9d,Mikey Moore,Tottenham,15,2.0,0.0,4.0,0.0,0.00,0.00
3,00963611,Konstantinos Mavropanos,West Ham,25,19.0,16.0,1500.0,16.7,0.07,0.01
4,00b77ecf,Nathan Fraser,Wolves,18,7.0,1.0,186.0,2.1,0.16,0.04
...,...,...,...,...,...,...,...,...,...,...
1304,fd45ca7b,Ryan Leonard,Millwall,31,35.0,31.0,2711.0,30.1,0.03,0.05
1305,fd985c15,Harry Pickering,Blackburn,24,36.0,35.0,3000.0,33.3,0.02,0.06
1306,fe558029,Mark Travers,Stoke City,24,13.0,13.0,1170.0,13.0,0.00,0.00
1307,ff9fcd4d,Alex Pritchard,Birmingham City,30,9.0,3.0,333.0,3.7,0.05,0.03


In [122]:
master = pd.read_csv("C:/Users/erknud3/fpl-optimization/model/data/Master.csv")
master = master.loc[master['24-25'].notna(), ['first_name', 'second_name', 'web_name', '24-25', 'fbref']]
master['24-25'] = master['24-25'].astype(int)
master = master.rename({'24-25': 'fpl_id'}, axis=1)

In [189]:
master.dtypes

first_name     object
second_name    object
web_name       object
fpl_id          int32
fbref          object
dtype: object

In [148]:
r = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
fpl_data = r.json()
elements = pd.DataFrame(fpl_data["elements"])
elements = elements[['id', 'team', 'element_type', 'now_cost', 'selected_by_percent']]
elements.loc[elements.element_type == 1, 'position'] ='GKP'
elements.loc[elements.element_type == 2, 'position'] ='DEF'
elements.loc[elements.element_type == 3, 'position'] ='MID'
elements.loc[elements.element_type == 4, 'position'] ='FWD'
elements = elements.rename({"selected_by_percent": "tsb"}, axis=1)
elements['now_cost'] = np.round(elements['now_cost'] / 10, 1)

In [170]:
f = requests.get("https://fantasy.premierleague.com/api/fixtures/")
fixtures = pd.DataFrame(f.json())
fixtures = fixtures[['event', 'team_h', 'team_a']]

In [230]:
fixtures

,event,team_h,team_a
0,1,14,9
1,1,10,12
2,1,1,20
3,1,8,5
4,1,15,17
...,...,...,...
375,38,15,8
376,38,16,6
377,38,17,1
378,38,18,5


In [242]:
fpl_players = players_df.merge(master, on='fbref', how='inner')

In [243]:
fpl_players = fpl_players.merge(elements, left_on='fpl_id', right_on='id', how='inner')

In [245]:
fpl_players = fpl_players[['fbref', 'fpl_id', 'Player', 'web_name', 'element_type', 'now_cost', 'tsb', 'position', 'Age', 'MP', 'Starts', 'Min', '90s', 'npxG', 'xAG']]

In [246]:
fpl_players = fpl_players.groupby(['fbref', 'fpl_id', 'Player', 'web_name', 'element_type', 'now_cost', 'tsb', 'position', 'Age']).agg({
    'MP': 'sum', 
    'Starts': 'sum', 
    'Min': 'sum', 
    '90s': 'sum',
    'npxG': 'mean', 
    'xAG': 'mean'
}).reset_index()

In [247]:
for col in ['MP', 'Starts', 'Min', '90s']:
    fpl_players[col] = fpl_players[col].astype(int)
    
for col in ['npxG', 'xAG']:
    fpl_players[col] = np.round(fpl_players[col].astype(float), 2)

In [248]:
fpl_players.sort_values(by= 'npxG' , ascending=False).head(20)

,fbref,fpl_id,Player,web_name,element_type,now_cost,tsb,position,Age,MP,Starts,Min,90s,npxG,xAG
262,95bd120d,165,Deivid Washington,Deivid,4,5.0,0.1,FWD,18,2,0,13,0,3.23,0.00
233,8450467d,260,Ali Al Hamadi,Al-Hamadi,4,5.0,0.4,FWD,21,14,1,293,3,0.99,0.11
28,123d2733,519,George Earthy,Earthy,3,4.5,0.2,MID,18,3,0,34,0,0.82,0.00
52,1f44ac21,351,Erling Haaland,Haaland,4,15.0,49.1,FWD,23,31,29,2552,28,0.81,0.15
474,f8eca1b6,65,Enes Ünal,Enes Ünal,4,5.5,0.1,FWD,26,16,2,328,3,0.80,0.28
139,4bcf39f6,229,Neal Maupay,Maupay,4,5.0,1.7,FWD,26,31,14,1432,15,0.77,0.11
144,4d77b365,316,Darwin Núñez,Darwin,4,7.5,6.2,FWD,24,36,22,2047,22,0.68,0.27
413,da2b8db3,276,Freddie Ladapo,Ladapo,4,4.5,2.1,FWD,30,17,2,350,3,0.67,0.05
369,c596fcb0,421,Callum Wilson,Wilson,4,7.0,0.2,FWD,31,20,9,991,11,0.65,0.05
425,dd745366,150,Andrey Santos,Andrey Santos,3,4.5,0.0,MID,19,1,0,8,0,0.64,0.00


In [249]:
player_csv_file_path = "C:/Users/erknud3/fpl-optimization/model/data/player_season_data.csv"

fpl_players.to_csv(player_csv_file_path, index=False)

In [219]:
# Step 1: Merge master_table with players_df using fbref id
player_stats = pd.merge(master, players_df, on='fbref', how='inner')

In [222]:
player_stats

,first_name,second_name,web_name,fpl_id,fbref,Player,Squad,Age,MP,Starts,Min,90s,npxG,xAG,id,team,element_type,now_cost,tsb,position
0,James,Milner,Milner,134,2f90f6b8,James Milner,Brighton,37,15.0,11.0,773.0,8.6,0.03,0.21,134,5,3,5.0,0.1,MID
1,James,Tarkowski,Tarkowski,236,15ea812b,James Tarkowski,Everton,30,38.0,38.0,3419.0,38.0,0.07,0.06,236,8,2,5.0,2.2,DEF
2,Ashley,Young,Young,238,be927d03,Ashley Young,Everton,38,31.0,27.0,2280.0,25.3,0.02,0.06,238,8,2,4.5,0.2,DEF
3,John,Ruddy,Ruddy,414,da8b19e2,John Ruddy,Birmingham City,36,44.0,44.0,3960.0,44.0,0.00,0.00,414,15,1,4.0,0.3,GKP
4,Łukasz,Fabiański,Fabianski,521,9328b835,Łukasz Fabiański,West Ham,38,10.0,7.0,721.0,8.0,0.00,0.00,521,19,1,4.0,12.7,GKP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Mark,O’Mahony,O’Mahony,138,a3afdd69,Mark O'Mahony,Brighton,18,3.0,1.0,63.0,0.7,0.08,0.00,138,5,4,4.5,0.5,FWD
511,Murillo,Santiago Costa dos Santos,Murillo,436,1704b0b8,Murillo,Nott'ham Forest,21,32.0,32.0,2789.0,31.0,0.06,0.04,436,16,2,4.5,0.6,DEF
512,Leon,Chiwome,Chiwome,539,a2852b4f,Leon Chiwome,Wolves,17,3.0,2.0,174.0,1.9,0.05,0.00,539,20,4,4.5,0.2,FWD
513,Enso,González,Gonzalez,547,7e5bebeb,Enso Gonzalez,Wolves,18,1.0,0.0,1.0,0.0,0.00,0.00,547,20,3,4.5,0.0,MID


In [253]:
# Step 1: Merge master_table with players_df using fbref id
player_stats = pd.merge(master, players_df, on='fbref', how='inner')

# Step 2: Merge with elements to get team_id for each player
player_stats = pd.merge(player_stats, elements, left_on='fpl_id', right_on='id', how='inner')

# Step 3: Merge player stats with fixtures table based on the team being in either team_h or team_a
fact_table_home = pd.merge(player_stats, fixtures, left_on='team', right_on='team_h', how='inner')
fact_table_home['home_or_away'] = 'H'
fact_table_home['team_facing'] = fact_table_home['team_a']

fact_table_away = pd.merge(player_stats, fixtures, left_on='team', right_on='team_a', how='inner')
fact_table_away['home_or_away'] = 'A'
fact_table_away['team_facing'] = fact_table_away['team_h']

# Combine the home and away matches into one fact table
fact_table = pd.concat([fact_table_home, fact_table_away], ignore_index=True)

# Cleanup to only keep necessary columns
fact_table = fact_table[['fbref', 'fpl_id', 'team', 'event', 'team_facing', 'home_or_away']]

In [255]:
fact_table_csv_file_path = "C:/Users/erknud3/fpl-optimization/model/data/fact_table.csv"

fact_table.to_csv(fact_table_csv_file_path, index=False)

In [254]:
fact_table.sort_values(by=['fpl_id','event'], ascending=True).head(40)

,fbref,fpl_id,team,event,team_facing,home_or_away
6840,6412cc03,1,1,1,20,H
16625,6412cc03,1,1,2,2,A
6841,6412cc03,1,1,3,5,H
16626,6412cc03,1,1,4,18,A
16627,6412cc03,1,1,5,13,A
6842,6412cc03,1,1,6,11,H
6843,6412cc03,1,1,7,17,H
16628,6412cc03,1,1,8,3,A
6844,6412cc03,1,1,9,12,H
16629,6412cc03,1,1,10,15,A
